In [37]:
import pandas as pd
import numpy as np

from pyscf import gto, scf, ao2mo


In [34]:
element = 'H'
url = f"https://www.molpro.net/info/basis.php?search=1&element={element}&basis=Dunning-DZP&print=1"
dfs = pd.read_html(url)

In [54]:
mol = gto.Mole()
mol.atom = f'''{element}  0 0 0'''
mol.charge = 0
mol.spin = 1
mol.unit = 'AU'


In [63]:
if False:
    mol.basis = {'H': [[0,
                        (19.2406000, 0.0328280),
                        (2.8992000, 0.2312080),
                        (0.6534000, 0.8172380),],
                       [0,
                        (0.1776000, 1.0000000),],
                       [1,
                        (1.0000000, 1.0000000),]],
    }


In [74]:
if True:
    basis = {}
    basis[element] = []
    for l in range(0,len(dfs)-1):
        #basis[element].extend([l, np.ndarray.tolist(dfs[l].values)])
        basis[element].append([l])
        for contraction in np.ndarray.tolist(dfs[l].values):
            for value in contraction:
                if value > 0.0:
                    basis[element].append(value)
    print(basis)
    
mol.basis = basis 


{'H': [[0], 19.2406, 0.032828, 2.8992, 0.231208, 0.6534, 0.817238, 0.1776, 1.0]}


In [73]:
mol.build()

IndexError: list index out of range

In [66]:
hf = scf.ROHF(mol).run()


converged SCF energy = -0.497637244627011
